In [ ]:
# Document loading, retrieval methods and text splitting
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Do imports
import os
import logging
import sys
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
# Enable tracing
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Load the Trivia Data from the CSV

In [4]:
# Use the LangChain CSV loader to load the trivia dataset from the CSV

from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path="test_data/test_question_set_cleaned.csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": ["category", "question", "answer"],
    },)
data = loader.load()

for record in data[:3]:
    print(record)

page_content='category: Category
question: Question
answer: Answer' metadata={'source': 'test_data/test_question_set_cleaned.csv', 'row': 0}
page_content='category: HISTORY
question: For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory
answer: Copernicus' metadata={'source': 'test_data/test_question_set_cleaned.csv', 'row': 1}
page_content='category: ESPN's TOP 10 ALL-TIME ATHLETES
question: No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves
answer: Jim Thorpe' metadata={'source': 'test_data/test_question_set_cleaned.csv', 'row': 2}


### Initialize the Vector Store with nomic-embed-text

In [5]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=data, embedding=local_embeddings)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'
load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'
load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'
load_verify_locat

### Add the LLM Model

In [6]:
# Add phi3:3.8b LLM model
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="phi3:3.8b",
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'
load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'
load_verify_locations cafile='/opt/anaconda3/envs/dev/lib/python3.10/site-packages/certifi/cacert.pem'


In [ ]:
# Test LLM model is working
response_message = model.invoke("What is the capital of France?")

print(response_message.content)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.smith.langchain.com:443
Starting new HTTPS connection (1): api.smith.langchain.com:443
DEBUG:httpcore.connection:connect_tcp.started host='127.0.0.1' port=11434 local_address=None timeout=None socket_options=None
connect_tcp.started host='127.0.0.1' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x11fbb7910>
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x11fbb7910>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
send_reque

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /info HTTP/11" 200 605
https://api.smith.langchain.com:443 "GET /info HTTP/11" 200 605
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 401 26
https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 401 26
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=626c1ab2-dced-4597-975f-4b2eea7db2ac,id=626c1ab2-dced-4597-975f-4b2eea7db2ac
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/x-ndjson'), (b'Date', b'Thu, 21 Nov 2024 21:02:50 GMT'), (b'Transfer-Encoding', b'chunked')])
receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type

In [8]:
# Test the retrieval chain is working
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)

# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What is the most popular category?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Thu, 21 Nov 2024 21:02:51 GMT'), (b'Transfer-Encoding', b'chunked')])
receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Thu, 21 Nov 2024 21:02:51 GMT'), (b'Transfer-Encoding', b'

DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 401 26
https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 401 26
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=626c1ab2-dced-4597-975f-4b2eea7db2ac,id=626c1ab2-dced-4597-975f-4b2eea7db2ac
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 401 26
https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 401 26
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=62bb0308-ce82

'Main themes in the retrieved docs include various cultural, professional and scientific aspects. Here are some summarized main points according to different categories mentioned in the documents:\n\n- Category EVERYBODY TALKS ABOUT IT... - This category focuses on common topics or issues that people frequently discuss. In this case, it is about a widely recognized tool for measuring and understanding sun exposure called "the UV index". The document explains its importance since June 28, 1994 when the National Weather Service started issuing it to rate solar radiation intensity in order to safeguard public health.\n  \n- Category VARIETY PACK - In this category are mentioned two distinct concepts: a slang term used by musicians and others for job opportunities or engagements, referred as "a gig", and also the name of an object (pronged spear) that is traditionally employed in fishing activities.\n  \n- Category CORN-UCOPIA - This category provides details about a specific type of liquo

In [9]:
# Basic RAG prompt - needs to be updated to use our RAG model and workflow / chain
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are a trivial chatbot. You will be asked questions and you will provide answers based on the context provided.
You will give the user a set of categories and ask them to choose one. You will then provide a question from that category.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

DEBUG:httpcore.connection:close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
DEBUG:httpcore.connection:connect_tcp.started host='127.0.0.1' port=11434 local_address=None timeout=None socket_options=None
connect_tcp.started host='127.0.0.1' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1041c1570>
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1041c1570>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
send_request_body.complete
DEBUG:httpcore.http11:receiv

"Task decomposition is a critical aspect in various fields, including artificial intelligence and human task management. Here are some general strategies for decomposing tasks effectively:\n\n1. Hierarchical decomposition - This method involves breaking down complex tasks into simpler subtasks recursively until each piece becomes manageable or trivial enough to perform independently. In this approach, a top-down strategy is often employed where the system starts with an overview of the task and progressively refines it by identifying smaller units within that scope.\n\n2. Bottom-up decomposition - This method begins at the most granular level possible in pursuit to identify subtasks before moving on to higher levels, effectively starting from small components working their way up through a hierarchy until reaching more significant tasks or goals. It can sometimes be useful when there is clear evidence of how individual pieces contribute towards completing an overall task without prior 